In [ ]:
!pip install --upgrade wget

In [ ]:
import os
import sys
import wget
filename = 'test.zip'
url = 'https://github.com/benoitdurand1/FSA2019/blob/master/test.zip?raw=true'
if not os.path.isfile( filename ): wget.download(url)
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [ ]:
cos_credentials = {
"apikey": "e8vBM2C1jpl54bv56pZVYRCA10zMwiW_dzy6TPXAI3TX",
  "cos_hmac_keys": {
    "access_key_id": "0cddd9bc4dcf4b2e9c8967da10e552c6",
    "secret_access_key": "725ca0c3efb0571bf3ad6d2ed053659a6a84f660d24b0ddf"
  },
  "endpoints": "https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:cloud-object-storage:global:a/700fedaab4a4e4956a2197be65fcc2c9:8f6631cb-8ffc-4170-9c60-d87088d7de72::",
  "iam_apikey_name": "auto-generated-apikey-0cddd9bc-4dcf-4b2e-9c89-67da10e552c6",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/700fedaab4a4e4956a2197be65fcc2c9::serviceid:ServiceId-6ee97b64-b8e7-44a5-b550-6b54ab23c0b7",
  "resource_instance_id": "crn:v1:bluemix:public:cloud-object-storage:global:a/700fedaab4a4e4956a2197be65fcc2c9:8f6631cb-8ffc-4170-9c60-d87088d7de72::"
}
print ("cos",cos_credentials )
service_endpoint = "https://s3.private.eu.cloud-object-storage.appdomain.cloud"

buckets = {
    "training": "bucket-training-gb-fsa",
    "results": "bucket-results-gb-fsa"
}

wml_credentials = {
    "apikey": "CyYcsgt4P_CA4nOEzESogvm6z2lObH6YjRtcHbtXlmX7",
  "iam_apikey_description": "Auto generated apikey during resource-key operation for Instance - crn:v1:bluemix:public:pm-20:us-south:a/700fedaab4a4e4956a2197be65fcc2c9:c0fd051b-0446-4442-94df-0c3fe39eb4a5::",
  "iam_apikey_name": "auto-generated-apikey-d438c762-030b-4883-9e03-3862bc5abdd2",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/700fedaab4a4e4956a2197be65fcc2c9::serviceid:ServiceId-7cd22c72-8c9d-46da-9cf7-93debc3ae4c5",
  "instance_id": "c0fd051b-0446-4442-94df-0c3fe39eb4a5",
  "password": "69df10f9-854f-40e4-906f-556133f9886a",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "d438c762-030b-4883-9e03-3862bc5abdd2"
}
print ("wml",wml_credentials )

In [ ]:
client = WatsonMachineLearningAPIClient(wml_credentials)
print("client", client)

In [ ]:
execution_command = "echo ${DATA_DIR}; echo ${RESULT_DIR}; export PYTHONPATH=$PYTHONPATH:${DATA_DIR}/models/:${DATA_DIR}/models/research/:${DATA_DIR}/models/research/slim/; python3 export_inference_graph.py --input_type=image_tensor --pipeline_config_path=${DATA_DIR}/models/research/object_detection/training/faster_rcnn_inception_v2_coco.config --trained_checkpoint_prefix=${DATA_DIR}/models/research/object_detection/training/model.ckpt-17818 --output_directory=${RESULT_DIR}; python3 object_detection_image.py"
print ("execution", execution_command)


In [ ]:
model_definition_metadata = {
    client.repository.DefinitionMetaNames.NAME: "repository-FSA",
    client.repository.DefinitionMetaNames.DESCRIPTION: "FSA MODEL",
    client.repository.DefinitionMetaNames.AUTHOR_NAME: "CEDRINE MADERA",
    client.repository.DefinitionMetaNames.FRAMEWORK_NAME: "tensorflow",
    client.repository.DefinitionMetaNames.FRAMEWORK_VERSION: "1.10",
    client.repository.DefinitionMetaNames.RUNTIME_NAME: "python",
    client.repository.DefinitionMetaNames.RUNTIME_VERSION: "3.5",
    client.repository.DefinitionMetaNames.EXECUTION_COMMAND: execution_command
}

In [ ]:
definition_details = client.repository.store_definition("test.zip", model_definition_metadata)
definition_uid = client.repository.get_definition_uid(definition_details)
print("The definition_uid is: {}".format(definition_uid))

In [ ]:
training_configuration_metadata = {
    client.training.ConfigurationMetaNames.NAME: "training-FSA", 
    client.training.ConfigurationMetaNames.AUTHOR_NAME: "CEDRINE MADERA",              
    client.training.ConfigurationMetaNames.DESCRIPTION: "FSA beer model",
    client.training.ConfigurationMetaNames.COMPUTE_CONFIGURATION: {"name": "k80"},
    client.training.ConfigurationMetaNames.TRAINING_DATA_REFERENCE: {
        "connection": {
            "endpoint_url": service_endpoint,
            "access_key_id": cos_credentials['cos_hmac_keys']['access_key_id'],
            "secret_access_key": cos_credentials['cos_hmac_keys']['secret_access_key']
        },
        "source": {
            "bucket": buckets['training'],
        },
        "type": "s3"
    },
    client.training.ConfigurationMetaNames.TRAINING_RESULTS_REFERENCE: {
        "connection": {
            "endpoint_url": service_endpoint,
            "access_key_id": cos_credentials['cos_hmac_keys']['access_key_id'],
            "secret_access_key": cos_credentials['cos_hmac_keys']['secret_access_key']
        },
        "target": {
            "bucket": buckets['results'],
        },
        "type": "s3"
    },
}
print("training:",training_configuration_metadata)

In [ ]:
training_run_details = client.training.run(definition_uid, training_configuration_metadata)


In [ ]:
training_run_guid_async = client.training.get_run_uid(training_run_details)
print("The training_run_guid_async is: {}".format(training_run_guid_async))

In [ ]:
client.training.monitor_logs(training_run_guid_async)

In [ ]:
saved_model_details = client.repository.store_model(training_run_guid_async, {"name": "My object detection model"})
print(client.repository.get_model_url(saved_model_details))
model_uid = client.repository.get_model_uid(saved_model_details)
print("Saved model uid: " + model_uid)

In [ ]:
deployment_details = client.deployments.create(model_uid, "My object detection deployment")
scoring_url = client.deployments.get_scoring_url(deployment_details)
print(scoring_url)

In [ ]:
# client.training.delete(training_run_guid_async)
# client.repository.delete(definition_uid)
# client.deployments.delete(deployments_uid)